In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib
import seaborn as sns 
import json 
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV 

%matplotlib inline

D:\Programs\Python\Python39\lib\site-packages\sklearn\utils\multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
D:\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
D:\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modif

In [2]:
df1 = pd.read_csv('us_2017_db.csv')
df1.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,3000.0,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,6750.0,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,30000.0,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,12500.0,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,22500.0,141000,Diesel,Individual,Manual,Second Owner


In [3]:
df2 = df1.drop(['name','seller_type'],axis=1).reindex(['transmission','fuel','owner','year','km_driven','selling_price'],axis=1)
df2.head()

,transmission,fuel,owner,year,km_driven,selling_price
0,Manual,Petrol,First Owner,2007,70000,3000.0
1,Manual,Petrol,First Owner,2007,50000,6750.0
2,Manual,Diesel,First Owner,2012,100000,30000.0
3,Manual,Petrol,First Owner,2017,46000,12500.0
4,Manual,Diesel,Second Owner,2014,141000,22500.0


In [4]:
df2['selling_price_usd'] = round(df2['selling_price'],2)


df3 = df2.drop('selling_price',axis=1)



In [5]:
df4 = df3
df4.fuel = df4.fuel.map(lambda x: x if x in ['Diesel','Petrol'] else 'Other')
df4.fuel.unique()

array(['Petrol', 'Diesel', 'Other'], dtype=object)

In [6]:
manual = df4[df4['transmission']=='Manual']
automatic = df4[df4['transmission']=='Automatic']

In [7]:
df5 = df4[~(df4['owner']=='Test Drive Car')]
df5.owner = df5.owner.map(lambda x: x if x in ['First Owner','Second Owner'] else 'Third Owner & Above')

df5['owner'].value_counts()

C:\Users\George\AppData\Local\Temp\ipykernel_10004\174906330.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5.owner = df5.owner.map(lambda x: x if x in ['First Owner','Second Owner'] else 'Third Owner & Above')


First Owner            2832
Second Owner           1106
Third Owner & Above     385
Name: owner, dtype: int64

In [8]:
km_mean = df5.groupby('year')['km_driven'].mean()

In [10]:
year_mean_manual = df5[df5['transmission']=='Manual'].groupby('year')['selling_price_usd'].mean()
year_mean_automatic = df5[df5['transmission']=='Automatic'].groupby('year')['selling_price_usd'].mean()

In [12]:
df5.groupby('transmission').agg(['mean','std','min','max']).applymap(lambda x: format(x,'.0f')).drop(['year'],axis=1)

C:\Users\George\AppData\Local\Temp\ipykernel_10004\2520389099.py:1: FutureWarning: ['fuel', 'owner'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df5.groupby('transmission').agg(['mean','std','min','max']).applymap(lambda x: format(x,'.0f')).drop(['year'],axis=1)


km_driven                      selling_price_usd               \
                  mean    std   min     max              mean    std   min   
transmission                                                                 
Automatic        49771  37676  1100  235000             70435  64313  3950   
Manual           68384  47115     1  806599             19892  13923  1000   

                      
                 max  
transmission          
Automatic     445000  
Manual         95000

In [13]:
def remove_outlier_km_driven(df):
    temp = pd.DataFrame()
    
    df_km_driven = df['km_driven']
    Q1 = df_km_driven.quantile(0.25)
    Q3 = df_km_driven.quantile(0.75)
    IQR = Q3 - Q1
    df_outlier = df_km_driven[(df_km_driven < (Q1 - 1.5 * IQR)) | (df_km_driven > (Q3 + 1.5 * IQR))]
    temp = pd.concat([temp,df_outlier])
        
    return df.drop(temp.index)

df6 = remove_outlier_km_driven(df5)

In [14]:
def remove_outlier_selling_price(df):
    temp = pd.DataFrame()
    for year in sorted(df.year.unique()):
        year_price_manual = df[(df['year']==year) & (df['transmission']=='Manual')]['selling_price_usd']
        manual_Q1 = year_price_manual.quantile(0.25)
        manual_Q3 = year_price_manual.quantile(0.75)
        manual_IQR = manual_Q3 - manual_Q1
        outlier_manual = year_price_manual[(year_price_manual < (manual_Q1 - 1.5 * manual_IQR)) | (year_price_manual > (manual_Q3 + 1.5 * manual_IQR))]
        temp = pd.concat([temp,outlier_manual])
        
        year_price_automatic = df[(df['year']==year) & (df['transmission']=='Automatic')]['selling_price_usd']
        automatic_Q1 = year_price_automatic.quantile(0.25)
        automatic_Q3 = year_price_automatic.quantile(0.75)
        automatic_IQR = automatic_Q3 - automatic_Q1
        outlier_automatic = year_price_automatic[(year_price_automatic < (automatic_Q1 - 1.5 * automatic_IQR)) | (year_price_automatic > (automatic_Q3 + 1.5 * automatic_IQR))]
        temp = pd.concat([temp,outlier_automatic])
    return df.drop(temp.index)

df7 = remove_outlier_selling_price(df6)

In [15]:
year_mean_manual = df7[df7['transmission']=='Manual'].groupby('year')['selling_price_usd'].mean()
year_mean_automatic = df7[df7['transmission']=='Automatic'].groupby('year')['selling_price_usd'].mean()

In [16]:

df8 = df7[df7['year']>2005]

In [17]:
year_mean_manual_price = df8[df8['transmission']=='Manual'].groupby('year')['selling_price_usd'].mean()
year_mean_automatic_price = df8[df8['transmission']=='Automatic'].groupby('year')['selling_price_usd'].mean()
year_mean_manual_km = df8[df8['transmission']=='Manual'].groupby('year')['km_driven'].mean()
year_mean_automatic_km = df8[df8['transmission']=='Automatic'].groupby('year')['km_driven'].mean()

In [18]:
def remove_outlier_last_year(df):
    temp = pd.DataFrame()
    years = sorted(df.year.unique())
    for i in range(1,len(years)):
        df_year = df[(df['year']==years[i])&(df['transmission']=='Manual')]
        last_mean_km_driven = df[(df['year']==years[i-1])&(df['transmission']=='Manual')]['km_driven'].mean()
        last_mean_selling_price = df[(df['year']==years[i-1])&(df['transmission']=='Manual')]['selling_price_usd'].mean() 
        df_outlier = df_year[(df_year['km_driven']>last_mean_km_driven)&(df_year['selling_price_usd']<last_mean_selling_price)]
        temp = pd.concat([temp,df_outlier])
        
        df_year = df[(df['year']==years[i])&(df['transmission']=='Automatic')]
        last_mean_km_driven = df[(df['year']==years[i-1])&(df['transmission']=='Automatic')]['km_driven'].mean()
        last_mean_selling_price = df[(df['year']==years[i-1])&(df['transmission']=='Automatic')]['selling_price_usd'].mean() 
        df_outlier = df_year[(df_year['km_driven']>last_mean_km_driven)&(df_year['selling_price_usd']<last_mean_selling_price)]
        temp = pd.concat([temp,df_outlier]) 
    return df.drop(temp.index)
    
df9 = remove_outlier_last_year(df8)

In [19]:
def remove_outlier_owner(df):
    temp = pd.DataFrame()
    for a in sorted(df.year.unique()):
        for b in ['Manual','Automatic']:
            df_year = df[(df['year']==a)&(df['transmission']==b)]
            second_mean = df_year[df_year['owner']=='Second Owner']['selling_price_usd'].mean()
            third_mean = df_year[df_year['owner']=='Third Owner & Above']['selling_price_usd'].mean()
            df_outlier = df_year[((df_year['owner']=='Second Owner')&(df_year['selling_price_usd']<third_mean)) | ((df_year['owner']=='First Owner')&(df_year['selling_price_usd']<second_mean))]
            temp = pd.concat([temp,df_outlier])
    return df.drop(temp.index)
    
df10 = remove_outlier_owner(df9)

In [20]:
def remove_outlier_last_year(df):
    temp = pd.DataFrame()
    for i in sorted(df.year.unique()):
        df_year = df[df['year']==i]
        km = df_year['km_driven']
        price = df_year['selling_price_usd']
        outlier = df_year[(df_year['km_driven']<km.mean()-km.std())&(df_year['selling_price_usd']<price.mean()-price.std()) | (df_year['km_driven']>km.mean()+km.std())&(df_year['selling_price_usd']>price.mean()+price.std())]
        temp = pd.concat([temp,outlier])

    return df.drop(temp.index)
    
df11 = remove_outlier_last_year(df10)

In [21]:
manual = df11[df11['transmission']=='Manual']
automatic = df11[df11['transmission']=='Automatic']

In [22]:
transmission = pd.get_dummies(df11['transmission'])
fuel = pd.get_dummies(df11['fuel'])
owner = df11['owner'].map(lambda x: 1 if x=='First Owner' else 2 if x=='Second Owner' else 3)

X = pd.concat([transmission,fuel,owner,df11.drop(['transmission','fuel','owner','selling_price_usd'],axis=1)],axis=1)
y = df11['selling_price_usd']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [24]:
lr = LinearRegression()
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

0.6539266868763558

In [25]:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

array([0.61890108, 0.60390357, 0.54983026, 0.67250382, 0.62429909])

### საუკეთესო მოდელი GridSearchCV მიხედვით

In [26]:
def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        },
        'random_forest': {
            'model': RandomForestRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'n_estimators': range(1,50,5)
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'მოდელი': algo_name,
            'ქულა': gs.best_score_,
            'პარამეტრები': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['მოდელი','ქულა','პარამეტრები'])

find_best_model_using_gridsearchcv(X,y)

D:\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:794: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
D:\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:794: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check 

D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note lin

D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note lin

D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note lin

D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note lin

D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note lin

D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note lin

,მოდელი,ქულა,პარამეტრები
0,linear_regression,0.613888,{'normalize': False}
1,lasso,0.613919,"{'alpha': 2, 'selection': 'cyclic'}"
2,decision_tree,0.841671,"{'criterion': 'mse', 'splitter': 'best'}"
3,random_forest,0.860324,"{'criterion': 'friedman_mse', 'n_estimators': 41}"


<div style="background-color: #A52A2A;padding: 20px;font-size: 16px;color:white;">
    შედეგებზე დაყრდნობით, გამოვიყენებთ RandomForestRegressor-ს
</div>	

In [27]:
model = RandomForestRegressor(criterion='mse',n_estimators=26)

In [28]:
model.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=26,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [29]:
def predict_price(transmission,fuel,previous_owner,year,km_driven):
    x = []
    x[:8] = np.zeros(8,dtype='int32')
    x[5] = previous_owner
    x[6] = year
    x[7] = km_driven
    
    transmission_index = np.where(X.columns==transmission)[0][0]
    fuel_index = np.where(X.columns==fuel)[0][0]
    
    if transmission_index>=0:
        x[transmission_index] = 1
    if fuel_index>=2:
        x[fuel_index] = 1
        
    return float(format(model.predict([x])[0],'.2f'))

In [30]:
predict_price('Manual','Diesel',1,2013,10000)

D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


23653.85

In [31]:
predict_price('Automatic','Diesel',1,2015,50000)

D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


50826.92

<div style="background-color: #A52A2A;padding: 20px;font-size: 16px;color:white;">
მოდელის შედეგი, იგივე გამოცდილება, შეინახება `pickle` ფორმატში. 
    მხატვრულად რომ ვთქვა - დაახლოებით კრიოგენულ მდგომარეობაში
</div>

In [32]:
with open('CRS_Model.pickle','wb') as f:
    pickle.dump(model,f)

In [33]:
with open('CRS_Model.pickle','rb') as file:
    mp = pickle.load(file)

In [34]:
mp.predict([[0,1,1,0,0,2,2017,30000]])

D:\Programs\Python\Python39\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


array([37071.27613782])

In [35]:
columns = {
    "svetebi": [col for col in X.columns]
}

with open("svetebi.json","w") as f:
    f.write(json.dumps(columns))